In [1]:
#Imoprt dependecies

import requests
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd

In [2]:
#Initiate browser
browser = Browser("chrome")

The chromedriver version (119.0.6045.105) detected in PATH at C:\Users\joseg\Documents\Bootcamp_2023\ChromeDriver\chromedriver.exe might not be compatible with the detected chrome version (120.0.6099.129); currently, chromedriver 120.0.6099.109 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry


In [8]:
# Visit the charley project site (specifically the part on missing people from Texas)
url = "https://charleyproject.org/case-searches/advanced-search?first-name=&middle-name=&last-name=&suffix=&sex=&missing-since=&missing-from-city=&missing-from-state=Texas&classification=&date-of-birth=&age=&height-and-weight=&distinguishing-chars=&clothing-jewelry-desc=&medical-conditions=&vehicle=&details-of-disappearance=&investigating-agency=&source-information="
browser.visit(url)

In [9]:
# Create a Beautiful Soup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [10]:
#Extract the links
html_links = soup.find_all("div", class_ = "case")
links = []

for i in html_links:
    missing_profile = i.find("a", href=True)
    if missing_profile is None:
        continue
    links.append(missing_profile["href"])

print(f"Number of missing people in Texas: {len(links)}")

#Initiate the dictionary
charley_porject_dictionary = {
    "Person Name": None,
    "Missing Since":None,
    "Missing From":None,  
    "Classification":None, 
    "Sex":None,
    "Race":None,  
    "Date of Birth":None,
    "Age":None,
    "Height and Weight":None,
}

#sample a link
links[0]

Number of missing people in Texas: 966


'https://charleyproject.org/case/angela-abbrederis'

In [11]:
#function to scrape each link
#The function  takes ina specific persons link and uses beatifulsoup to 
#parse through the html then finds and reurns the name, age, sex ect from the individual links.
#If there is missing data from the website the except will place a N/A in stead

def scrape_person_info(person_url):

    browser.visit(person_url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    try:
        person_name = (soup.find("h1", class_='title entry-title is-1').text.strip())
    except:
        person_name = ("N/A")
    try:
        missing_since = (soup.find('strong', string='Missing Since').next_sibling.replace("\t", "").replace("\n", ""))
    except:
         missing_since = ("N/A")
    try:
        missing_from = (soup.find('strong', string='Missing From').next_sibling.replace("\t", "").replace("\n", ""))
    except:
        missing_from = ("N/A")
    try:
        classification = (soup.find('strong', string='Classification').next_sibling.replace("\t", "").replace("\n", ""))
    except:
        classification = ("N/A")
    try:
        sex = (soup.find('strong', string='Sex').next_sibling.replace("\t", "").replace("\n", ""))
    except:
        sex = ("N/A")
    try:
        race = (soup.find('strong', string='Race').next_sibling.replace("\t", "").replace("\n", ""))
    except:
        race = ("N/A")
    try:
        date_of_birth = (soup.find('strong', string='Date of Birth').next_sibling.replace("\t", "").strip(" ").replace("\n", ""))
    except:
        date_of_birth = ("N/A")
    try:
        age = (soup.find('strong', string='Age').next_sibling.replace("\t", "").strip(" ").replace("\n", ""))
    except:
        age = ("N/A")
    try:
        height_and_weight = (soup.find('strong', string='Height and Weight').next_sibling.replace("\t", "").replace("\n", ""))
    except:
        height_and_weight = ("N/A")
        
    return(person_name, missing_since, missing_from, classification, sex, race, date_of_birth, age, height_and_weight)


In [13]:
#Create list to keep missing persons info
person_name_list = []
missing_since_list = []
missing_from_list = []
classification_list = []
sex_list = []
race_list = []
date_of_birth_list = []
age_list = []
height_and_weight_list = []

#Loop to get the desired number of data   
#This can be updated to get data from all missing cases from Texas
#for link in links: then replace links(i) with link
for link in links:
    
    scraped_data = scrape_person_info(link)
    person_name_list.append(scraped_data[0])
    missing_from_list.append(scraped_data[1])
    missing_since_list.append(scraped_data[2])
    classification_list.append(scraped_data[3])
    sex_list.append(scraped_data[4])
    race_list.append(scraped_data[5])
    date_of_birth_list.append(scraped_data[6])
    age_list.append(scraped_data[7])
    height_and_weight_list.append(scraped_data[8])
    
#Place the data in the dictionary   
charley_porject_dictionary["Person Name"] = person_name_list
charley_porject_dictionary["Missing From"] = missing_since_list
charley_porject_dictionary["Missing Since"] = missing_from_list
charley_porject_dictionary["Classification"] = classification_list
charley_porject_dictionary["Sex"] = sex_list
charley_porject_dictionary["Race"] = race_list
charley_porject_dictionary["Date of Birth"] = date_of_birth_list
charley_porject_dictionary["Age"] = age_list
charley_porject_dictionary["Height and Weight"] = height_and_weight_list  


In [14]:
missing_df = pd.DataFrame(data = charley_porject_dictionary)


,Person Name,Missing Since,Missing From,Classification,Sex,Race,Date of Birth,Age,Height and Weight
0,Angela Abbrederis,04/02/1996,"Dallas, Texas",Endangered Missing,Female,"Asian, Biracial, White",10/25/1968 (55),27 years old,"5'7 - 5'9, 115 - 130 pounds"
1,Guadalupe Acevedo,12/08/2004,"Grand Prairie, Texas",Missing,Male,Hispanic,12/13/1953 (70),50 years old,"5'8, 150 pounds"
2,Brent Marshall Acomb,01/24/2014,"Houston, Texas",Endangered Missing,Male,White,12/15/1959 (64),54 years old,"5'10 - 6'0, 165 pounds"
3,Debra Marie Adams,03/01/1989,"Abilene, Texas",Missing,Female,White,11/14/1967 (56),22 years old,"5'5, 110 pounds"
4,Michael Jefferson Adams,06/08/1987,"Abilene, Texas",Endangered Missing,Male,White,03/28/1969 (54),18 years old,"6'0 - 6'1, 145 - 150 pounds"
...,...,...,...,...,...,...,...,...,...
961,Marcos Arrellano Zavala,08/31/1998,"Willis, Texas",Endangered Missing,Male,Hispanic,10/07/1975 (48),22 years old,"5'10 - 6'0, 200 - 230 pounds"
962,Wensheng Zheng,08/17/1999,"Houston, Texas",Missing,Male,Asian,07/10/1968 (55),31 years old,"5'7, 145 pounds"
963,John Edward Zimmerman II,04/09/2005,"Freeport, Texas",Lost/Injured Missing,Male,White,08/18/1953 (70),51 years old,"5'10, 160 pounds"
964,Michael Jerome Zipfel,01/06/2005,"Coppell, Texas",Endangered Missing,Male,White,10/26/1958 (65),46 years old,"5'9, 130 - 140 pounds"


In [16]:
missing_df.to_csv("Missing.csv")